In [103]:
import pandas as pd
import numpy as np
from model import get_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU

In [104]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [105]:
# 1948-2000
slp_s,  slp_e  = 0,  53
uwnd_s, uwnd_e = 0,  53
sst_s,  sst_e  = 94, 147
rain_s, rain_e = 47, 100

In [106]:
import glob

slp_path  = "./Data/slp/*"
sst_path  = "./Data/sst/*"
uwnd_path = "./Data/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

In [107]:
# Data form 1948-2000
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:90]))
rain_fall_data = list(rain['Jun-Sep'][rain_s:])

In [108]:
# Extracing Data
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd= slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd= pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e], feature_uwnd[uwnd_s:uwnd_e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)
x_slp = np.array(combine_slp)
x_sst = np.array(combine_sst)
x_uwnd = np.array(combine_uwnd)
x = np.concatenate((x_slp, x_sst, x_uwnd), axis=1)
x.shape, x_slp.shape, x_sst.shape, x_uwnd.shape

((636, 713), (636, 324), (636, 65), (636, 324))

In [109]:
model_1 = get_model([713, 47])
model_1.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_22 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_14 (ReLU)             (None, 713)               0         
                                                                 
 dense_23 (Dense)            (None, 47)                33558     
                                                                 
 dense_24 (Dense)            (None, 713)               34224     
                                                                 
Total params: 576,864
Trainable params: 576,864
Non-trainable params: 0
_________________________________________________________________


In [110]:
# fit the autoencoder model to reconstruct input
history = model_1.fit(x, x, epochs=1600, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/1600
40/40 - 1s - loss: 15639.4688 - val_loss: 11005.8545 - 587ms/epoch - 15ms/step
Epoch 2/1600
40/40 - 0s - loss: 9736.1152 - val_loss: 8329.5811 - 186ms/epoch - 5ms/step
Epoch 3/1600
40/40 - 0s - loss: 7593.2012 - val_loss: 6666.2588 - 201ms/epoch - 5ms/step
Epoch 4/1600
40/40 - 0s - loss: 6316.0483 - val_loss: 5737.5552 - 214ms/epoch - 5ms/step
Epoch 5/1600
40/40 - 0s - loss: 5536.4927 - val_loss: 5084.1948 - 233ms/epoch - 6ms/step
Epoch 6/1600
40/40 - 0s - loss: 4969.3013 - val_loss: 4611.3281 - 232ms/epoch - 6ms/step
Epoch 7/1600
40/40 - 0s - loss: 4571.1289 - val_loss: 4295.0386 - 217ms/epoch - 5ms/step
Epoch 8/1600
40/40 - 0s - loss: 4275.8740 - val_loss: 4049.5295 - 194ms/epoch - 5ms/step
Epoch 9/1600
40/40 - 0s - loss: 4014.0090 - val_loss: 3787.0266 - 200ms/epoch - 5ms/step
Epoch 10/1600
40/40 - 0s - loss: 3798.4685 - val_loss: 3608.0051 - 199ms/epoch - 5ms/step
Epoch 11/1600
40/40 - 0s - loss: 3628.6726 - val_loss: 3450.8188 - 192ms/epoch - 5ms/step
Epoch 12/1600
40

In [111]:
model_2 = get_model([713, 49, 29])
model_2.summary()

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_25 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_15 (ReLU)             (None, 713)               0         
                                                                 
 dense_26 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_16 (ReLU)             (None, 49)                0         
                                                                 
 dense_27 (Dense)            (None, 29)                1450      
                                                                 
 dense_28 (Dense)            (None, 49)                147

In [112]:
weight_transfer_layer_model_2 = [1,2,3,8]
weight_transfer_from_layer_model_1 = [1,2,3,4]

for i in range(len(weight_transfer_layer_model_2)):
  model_2_index = weight_transfer_layer_model_2[i]
  model_1_index = weight_transfer_from_layer_model_1[i]
  model_2.layers[model_2_index] = model_1.layers[model_1_index]
  model_2.layers[model_2_index].trainable = False

In [113]:
model_2.summary()

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_25 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_15 (ReLU)             (None, 713)               0         
                                                                 
 dense_26 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_16 (ReLU)             (None, 49)                0         
                                                                 
 dense_27 (Dense)            (None, 29)                1450      
                                                                 
 dense_28 (Dense)            (None, 49)                147

In [114]:
# fit the autoencoder model to reconstruct input
history = model_2.fit(x, x, epochs=1600, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/1600
40/40 - 0s - loss: 16952.6406 - val_loss: 13839.3125 - 500ms/epoch - 12ms/step
Epoch 2/1600
40/40 - 0s - loss: 12513.4639 - val_loss: 11448.1514 - 155ms/epoch - 4ms/step
Epoch 3/1600
40/40 - 0s - loss: 11059.9238 - val_loss: 10435.0986 - 163ms/epoch - 4ms/step
Epoch 4/1600
40/40 - 0s - loss: 10130.5713 - val_loss: 9574.5635 - 161ms/epoch - 4ms/step
Epoch 5/1600
40/40 - 0s - loss: 9341.6328 - val_loss: 8826.2168 - 156ms/epoch - 4ms/step
Epoch 6/1600
40/40 - 0s - loss: 8600.8262 - val_loss: 8123.6304 - 161ms/epoch - 4ms/step
Epoch 7/1600
40/40 - 0s - loss: 7953.6807 - val_loss: 7540.0376 - 186ms/epoch - 5ms/step
Epoch 8/1600
40/40 - 0s - loss: 7441.3154 - val_loss: 7067.5854 - 175ms/epoch - 4ms/step
Epoch 9/1600
40/40 - 0s - loss: 7020.7988 - val_loss: 6773.8726 - 174ms/epoch - 4ms/step
Epoch 10/1600
40/40 - 0s - loss: 6741.6558 - val_loss: 6472.9648 - 157ms/epoch - 4ms/step
Epoch 11/1600
40/40 - 0s - loss: 6467.2710 - val_loss: 6271.0264 - 161ms/epoch - 4ms/step
Epoch 12/16

In [115]:
model_3 = get_model([713, 49, 29, 9])
model_3.summary()

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_30 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_18 (ReLU)             (None, 713)               0         
                                                                 
 dense_31 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_19 (ReLU)             (None, 49)                0         
                                                                 
 dense_32 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_20 (ReLU)             (None, 29)                0  

In [116]:
weight_transfer_layer_model_3 = [1, 2, 3, 4, 5, 10, 11, 12]
weight_transfer_from_layer_model_2 = [1, 2, 3, 4, 5, 6, 7, 8]

for i in range(len(weight_transfer_layer_model_3)):
  model_3_index = weight_transfer_layer_model_3[i]
  model_2_index = weight_transfer_from_layer_model_2[i]
  model_3.layers[model_3_index] = model_2.layers[model_2_index]
  model_3.layers[model_3_index].trainable = False

In [117]:
model_3.summary()

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_30 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_18 (ReLU)             (None, 713)               0         
                                                                 
 dense_31 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_19 (ReLU)             (None, 49)                0         
                                                                 
 dense_32 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_20 (ReLU)             (None, 29)                0  

In [118]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(x, x, epochs=2000, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/2000
40/40 - 1s - loss: 16843.2891 - val_loss: 14243.5332 - 679ms/epoch - 17ms/step
Epoch 2/2000
40/40 - 0s - loss: 13142.9492 - val_loss: 12044.8984 - 159ms/epoch - 4ms/step
Epoch 3/2000
40/40 - 0s - loss: 11715.2754 - val_loss: 11260.2441 - 164ms/epoch - 4ms/step
Epoch 4/2000
40/40 - 0s - loss: 11128.0918 - val_loss: 10666.5166 - 169ms/epoch - 4ms/step
Epoch 5/2000
40/40 - 0s - loss: 10544.1680 - val_loss: 10205.2695 - 155ms/epoch - 4ms/step
Epoch 6/2000
40/40 - 0s - loss: 10051.9990 - val_loss: 9726.3965 - 159ms/epoch - 4ms/step
Epoch 7/2000
40/40 - 0s - loss: 9686.5107 - val_loss: 9362.2119 - 166ms/epoch - 4ms/step
Epoch 8/2000
40/40 - 0s - loss: 9311.9316 - val_loss: 9055.5439 - 152ms/epoch - 4ms/step
Epoch 9/2000
40/40 - 0s - loss: 9042.5830 - val_loss: 8846.3662 - 164ms/epoch - 4ms/step
Epoch 10/2000
40/40 - 0s - loss: 8881.1191 - val_loss: 8665.8896 - 165ms/epoch - 4ms/step
Epoch 11/2000
40/40 - 0s - loss: 8718.8311 - val_loss: 8579.3438 - 151ms/epoch - 4ms/step
Epoch 1

In [119]:
model_final = get_model([713, 49, 29, 9])
for i in range(13):
    model_final.layers[i] = model_3.layers[i]
model_final.summary()

Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_37 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_23 (ReLU)             (None, 713)               0         
                                                                 
 dense_38 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_24 (ReLU)             (None, 49)                0         
                                                                 
 dense_39 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_25 (ReLU)             (None, 29)                0  

In [120]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/1000
40/40 - 1s - loss: 17125.3984 - val_loss: 14172.6240 - 552ms/epoch - 14ms/step
Epoch 2/1000
40/40 - 0s - loss: 13250.5830 - val_loss: 12368.0830 - 160ms/epoch - 4ms/step
Epoch 3/1000
40/40 - 0s - loss: 11956.5664 - val_loss: 11461.5586 - 170ms/epoch - 4ms/step
Epoch 4/1000
40/40 - 0s - loss: 11402.8506 - val_loss: 11145.2529 - 169ms/epoch - 4ms/step
Epoch 5/1000
40/40 - 0s - loss: 11118.8770 - val_loss: 10869.7002 - 156ms/epoch - 4ms/step
Epoch 6/1000
40/40 - 0s - loss: 10787.4658 - val_loss: 10457.4717 - 168ms/epoch - 4ms/step
Epoch 7/1000
40/40 - 0s - loss: 10405.5361 - val_loss: 10113.5234 - 159ms/epoch - 4ms/step
Epoch 8/1000
40/40 - 0s - loss: 10116.2295 - val_loss: 9856.9658 - 164ms/epoch - 4ms/step
Epoch 9/1000
40/40 - 0s - loss: 9779.2031 - val_loss: 9558.3975 - 167ms/epoch - 4ms/step
Epoch 10/1000
40/40 - 0s - loss: 9544.8105 - val_loss: 9309.6699 - 153ms/epoch - 4ms/step
Epoch 11/1000
40/40 - 0s - loss: 9364.1611 - val_loss: 9181.8320 - 164ms/epoch - 4ms/step
Epo

In [121]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[1].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final3.trainable = False

model_final4 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final4.trainable = False

models = [model_final1, model_final2, model_final3, model_final4]
# models = [model_final1, model_final4]

In [122]:
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
    new_data = []
    for i in range(len(data)):
      d = data.iloc[i]
      new_data.append(list(d[1:]))
    return np.array(new_data)

# to combine all the features together
def predictor(month,model_final):
    month_path_slp, month_path_sst, month_path_uwnd= slp_months[month], sst_months[month], uwnd_months[month] 
    month_data_slp ,month_data_sst, month_data_uwnd= pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
  
    feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
    feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:-1], feature_sst[sst_s:], feature_uwnd[uwnd_s:-1]
    
    feature = np.concatenate((feature_slp, feature_sst, feature_uwnd), axis=1)

    pred = np.array(feature)
    pred_m = model_final(pred)
    pred_f = pred_m.numpy()
    return pred_f

In [123]:
from sklearn.svm import SVR
from scipy.stats import pearsonr
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_f,w_start,w_end):
    cor_list = []
    for i in range(len(pred_f[0])):
        score = []
        
        for j in range(w_start,w_end):
            score.append(pred_f[j][i])
        # print(len(range(w_start,w_end)), rain_fall_data[w_end-1], len(score))
        corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
        cor_list.append(corr)
      # print(corr)
    list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
    list2=sorted(list1, key=lambda x: x[1],reverse=True)
    top_feature_index = []
    #here i am getting the top features index
    for i in range(top):
        index = list2[i][0]
        top_feature_index.append(index)
    # Here i am getting all the top features
    predictor = []
    for i in range(len(pred_f)):
        temp = []
        for j in top_feature_index:
          # print(i,"  ",j)
            feature = pred_f[i][j]
            temp.append(feature)
        predictor.append(temp)
    # print(list2)
    return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in models:
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b) 
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [124]:
months_comb = [[4]]
top = 9
# models = [model_final3 ]
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print(" Month combination = ",month,"    top = ",i,"   min and max",left,"   ",right)

Window size =  10    plcc -0.3186678396616972
Window size =  11    plcc -0.46342746265087986
Window size =  12    plcc -0.49867054296636604
Window size =  13    plcc -0.35662242471383776
Window size =  14    plcc -0.6365178625977621
Window size =  15    plcc -0.4170048131065795
Window size =  16    plcc -0.23624794551186853
Window size =  17    plcc -0.425429839163064
Window size =  18    plcc -0.49735052695574006
Window size =  19    plcc -0.353685821813771
Window size =  20    plcc -0.4741177288191444
Window size =  21    plcc -0.31923897335107787
Window size =  22    plcc -0.4933500793385104
Window size =  23    plcc -0.3358984027065686
Window size =  24    plcc -0.21467471109596242
Window size =  25    plcc -0.31097273456793667
Window size =  26    plcc -0.21469281491791892
Window size =  27    plcc -0.29579136744396606
Window size =  28    plcc -0.2877054971380277
Window size =  29    plcc -0.3351587529884349
Window size =  30    plcc -0.134179168734517
Window size =  31    plcc -